# Transferência de aprendizado com Hugging Face

Um curso preparatório está buscando alternativas para que os alunos possam praticar mais a redação de textos.

Atualmente o curso preparatório consegue fazer a correção de apenas uma redação por semana para cada aluno e por conta disso, os alunos não se sentem motivados a fazer mais redações, uma vez que não terão ideia de qual seria a nota recebida nas redações extras que não serão corrigidas pelo curso.

Nosso papel nesse projeto é construir um **aplicativo de IA** para atribuir uma nota a uma redação, para que os alunos possam praticar quantas vezes quiserem e saberem se estão indo bem nos estudos.

## Explorando modelo pré-treinado

### Utilizando o hugging face pipeline

Para construir o aplicativo, precisamos de algo que consiga interpretar os textos e atribuir uma nota para a redação. Vamos utilizar um modelo pré-treinado da plataforma Hugging Face para tentar solucionar nosso problema.

Como a nossa tarefa é bem específica, vamos utilizar um modelo zero-shot que consegue fazer a classificação de um texto a partir de labels personalizadas.

Modelo zero-shot em português:

- https://huggingface.co/Mel-Iza0/zero-shot

In [1]:
!pip install -q transformers

In [2]:
from transformers import pipeline

In [3]:
classificador = pipeline("zero-shot-classification", model="Mel-Iza0/zero-shot", tokenizer="Mel-Iza0/zero-shot")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/407 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [4]:
texto = 'Com o início da era digital, a capacidade de transmissão de informações cresceu apressuradamente, o que facilitou o contato com diversos assuntos, dentre eles a educação sexual . Entretanto, surgiram paralelamente algumas questões, das quais se destacam a preocupação com o momento adequado do ingresso do tema a vida do estudante, assim como de maneira antagônica, o aumento de casos de DST´S\\xa0\\xa0e gravidez indesejada nesse período, a qual leva a um maior questionamento sobre o começo desta pauta., A falta de comunicação sobre a sexualidade entre jovens no Brasil acarreta muitas das vezes\\xa0na inserção desses em um meio repleto de dúvidas, gerando a ocorrência de doenças sexualmente transmissíveis e de gravidez precoce. Com base nisso, muitos adolescentes buscam compreender melhor essas questões na internet, local onde se podem encontrar notícias falsas ou inadequadas para seu desenvolvimento, impedindo assim a correta compreensão do assunto, assim como a responsabilidade imposta por ele., Por outro lado, o diálogo em relação à sexualidade e seus tópicos é um tabu para pais e professores, que se sentem desorientados sobre a devida hora e os devidos critérios a serem tratados com os filhos e alunos, dificultando com que esses esclareçam suas dúvidas e entenda de maneira correta, o que levaria a conscientização da seriedade dessa discussão., Em virtude do que foi mencionado, as indagações a respeito divide várias opiniões e reflexões acima do que deve ser feito. É de extrema importância o Ministério da Educação, em parceria com o Ministério da Cidadania, implantar a educação sexual na matriz curricular estudantil dos jovens, através de aulas elaboradas e destinadas ao esclarecimento de perguntas, assim como palestras e programas com a intenção de propagar o conteúdo aos estudantes, contando com o suporte dos pais, funcionários e encarregados da rede de ensino no país, para que seja realizado constantemente.'

In [5]:
classificador(texto, candidate_labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Com o início da era digital, a capacidade de transmissão de informações cresceu apressuradamente, o que facilitou o contato com diversos assuntos, dentre eles a educação sexual . Entretanto, surgiram paralelamente algumas questões, das quais se destacam a preocupação com o momento adequado do ingresso do tema a vida do estudante, assim como de maneira antagônica, o aumento de casos de DST´S\\xa0\\xa0e gravidez indesejada nesse período, a qual leva a um maior questionamento sobre o começo desta pauta., A falta de comunicação sobre a sexualidade entre jovens no Brasil acarreta muitas das vezes\\xa0na inserção desses em um meio repleto de dúvidas, gerando a ocorrência de doenças sexualmente transmissíveis e de gravidez precoce. Com base nisso, muitos adolescentes buscam compreender melhor essas questões na internet, local onde se podem encontrar notícias falsas ou inadequadas para seu desenvolvimento, impedindo assim a correta compreensão do assunto, assim como a responsabil

# Realizando o deploy com gradio

Para que o aplicativo fique completo, podemos utilizar o gradio, uma ferramenta que permite interação de forma rápida e eficientes a modelos de IA

In [6]:
!pip install -q gradio

In [7]:
import gradio as gr

In [8]:
def mostrar_resultado(texto):
    return classificador(texto, candidate_labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])['labels'][0]

In [9]:
app = gr.Interface(
    fn=mostrar_resultado,
    inputs=["text"],
    outputs=["text"]
)

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6a41cec2f485b7aec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Aula 02 : Preparando dados de texto

# Carregando a base de dados

Como nossa tarefa é muito específica, o modelo zero-shot não vai lidar muito bem com a situação. O que podemos fazer é explorar uma base de dados que contém redações e suas respectivas notas e criar um modelo que fará a atribuição na nota com base no texto.

Tarefas de NLP são muito complexas e criar um modelo do zero a partir de poucos dados se torna ineficiente. O que vamos fazer é utilizar um modelo pré-treinado a partir de um corpus textual em português e transferir o conhecimento desse modelo para um novo modelo que vai se especializar na tarefa de atribuir notas as redações. Essa tarefa é conhecida como **Trasfer** **Learning** ou **transferência de aprendizado**.

In [10]:
!pip install -q datasets

In [11]:
from datasets import load_dataset

In [13]:
dados_redacoes = load_dataset("csv", data_files='/content/redacoes.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
dados_redacoes

DatasetDict({
    train: Dataset({
        features: ['essay', 'score'],
        num_rows: 4570
    })
})

In [16]:
dados_redacoes['train'].features

{'essay': Value('string'), 'score': Value('int64')}

In [17]:
dados_redacoes['train']['essay'][0]

'A convivência com o ensino em uma escola pública , nos mostra \\xa0como a reforma do ensino médio é necessária, com o ensino precário, escolas quase desabando, falta de educadores,a formação de estudantes fica a \\xa0cada dia mais difícil., Atualmente o índice de jovens que procuram uma educação de qualidade é maior do que em todos os séculos , temos em vista que todos procuram a melhoria, tanto na estrutura da instituição como no ensino, muitos sonham em ser: médicos, professores,advogados, contadores, e assim sucessivamente., A solução seria a constatação de uma lei em prol dos estudantes, precisamos ser ouvidos, a voz dos jovens é a voz do futuro, queremos o ensino superior, queremos oportunidades em escolas públicas, escolas bem estruturada e profissionais competentes., Creio \\xa0que a reforma deveria ocorrer, pois o intuito é a melhoria (Que tipo de melhoria?,\\xa0assim todos os jovens teriam a oportunidade de serem profissionais de sucesso. O incentivo ( aos jovens é o que torn

In [18]:
dados_redacoes['train']['score'][0]

4

In [19]:
dados_redacoes['train'].to_pandas()

,essay,score
0,A convivência com o ensino em uma escola públi...,4
1,O Brasil possui aproximadamente 425 mil polici...,5
2,"O esporte é muito importante, as Olimpíadas, u...",8
3,"O conceito de ""banalidade do mal"" da filósofa ...",8
4,Um dos assuntos que mais geram discussões atua...,5
...,...,...
4565,"Com o início da era digital, a capacidade de t...",6
4566,"O feminicídio é o assassinato de uma mulher, a...",4
4567,Foram vrias as conquistas que as mulheres bras...,5
4568,Segundo o psicoterapeuta Ivan Carpelatto: “a v...,9


# Separando os dados

Vamos fazer uma separação dos dados de treino e teste para validação do modelo de ML e evitar o teste em dados que foram usados no aprendizado do modelo.

In [20]:
treino_teste = dados_redacoes['train'].train_test_split(test_size=0.2, shuffle=False)
treino_teste

DatasetDict({
    train: Dataset({
        features: ['essay', 'score'],
        num_rows: 3656
    })
    test: Dataset({
        features: ['essay', 'score'],
        num_rows: 914
    })
})

In [21]:
from datasets import DatasetDict

In [22]:
dados_redacoes = DatasetDict({
    'treino': treino_teste['train'],
    'teste': treino_teste['test']
})

In [23]:
dados_redacoes

DatasetDict({
    treino: Dataset({
        features: ['essay', 'score'],
        num_rows: 3656
    })
    teste: Dataset({
        features: ['essay', 'score'],
        num_rows: 914
    })
})

# Tokenizando dados textuais

Para trabalhar com dados textuais, precisamos transformar as palavras em números que podem ser entendidos pelo computador. Vamos utilizar uma arquitetura específica para o modelo de ML que é chamada **Transformers**. Nessa arquitetura os tokens tem uma representação em ids e uma camada de atenção, que permite o modelo compreender melhor o sentido da palavra dentro do contexto da sentença.

Vamos utilizar um modelo pré-treinado que já possui armazenado toda a etapa de transformação do texto em tokens, portanto podemos importar o tokenizador do modelo que fará todo o processo dos dados de texto para o formato adequado.

Modelo de base para transferência de aprendizado e tokenização
*   https://huggingface.co/Geotrend/distilbert-base-pt-cased



In [24]:
checkpoint_modelo = "Geotrend/distilbert-base-pt-cased"

In [25]:
from transformers import AutoTokenizer

In [26]:
tokenizador = AutoTokenizer.from_pretrained(checkpoint_modelo)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [27]:
texto = dados_redacoes['treino']['essay'][0]

In [28]:
tokens = tokenizador.tokenize(texto)
print(tokens)

['A', 'con', '##viv', '##ência', 'com', 'o', 'ensino', 'em', 'uma', 'escola', 'pública', ',', 'nos', 'mostra', '\\', 'xa', '##0', '##com', '##o', 'a', 'reforma', 'do', 'ensino', 'médio', 'é', 'ne', '##ces', '##s', '##ária', ',', 'com', 'o', 'ensino', 'pre', '##c', '##ário', ',', 'escolas', 'quase', 'desa', '##band', '##o', ',', 'falta', 'de', 'edu', '##cado', '##res', ',', 'a', 'formação', 'de', 'estudantes', 'fica', 'a', '\\', 'xa', '##0', '##cada', 'dia', 'mais', 'difícil', '.', ',', 'Atualmente', 'o', 'índice', 'de', 'jovens', 'que', 'procura', '##m', 'uma', 'educação', 'de', 'qualidade', 'é', 'maior', 'do', 'que', 'em', 'todos', 'os', 'séculos', ',', 'tem', '##os', 'em', 'vista', 'que', 'todos', 'procura', '##m', 'a', 'melhor', '##ia', ',', 'tanto', 'na', 'estrutura', 'da', 'instituição', 'como', 'no', 'ensino', ',', 'muitos', 'son', '##ham', 'em', 'ser', ':', 'médicos', ',', 'professore', '##s', ',', 'ad', '##vog', '##ados', ',', 'conta', '##dores', ',', 'e', 'assim', 'su', '##ces

In [30]:
ids = tokenizador.convert_tokens_to_ids(tokens)
print(ids)

[46, 270, 24522, 2475, 303, 91, 13849, 348, 469, 5148, 5155, 25, 1125, 4201, 73, 5308, 759, 5563, 231, 77, 8008, 246, 13849, 19640, 144, 539, 2597, 206, 4354, 25, 303, 91, 13849, 1480, 415, 2854, 25, 16687, 8774, 3151, 4948, 231, 25, 5988, 203, 5725, 8294, 920, 25, 77, 11569, 203, 22157, 13510, 77, 73, 5308, 759, 6073, 608, 578, 7859, 27, 25, 12242, 91, 20373, 203, 18423, 220, 21237, 244, 469, 17937, 203, 16944, 144, 2695, 246, 220, 348, 1692, 463, 16426, 25, 1802, 383, 348, 3416, 220, 1692, 21237, 244, 77, 5915, 359, 25, 1813, 230, 9885, 240, 21600, 314, 286, 13849, 25, 6455, 385, 1948, 348, 510, 39, 17306, 25, 17575, 206, 25, 712, 22852, 2092, 25, 5719, 2669, 25, 81, 4832, 292, 2597, 12125, 576, 27, 25, 46, 2996, 10877, 4245, 77, 10673, 15832, 203, 469, 2900, 348, 955, 258, 441, 22157, 25, 15218, 2041, 510, 465, 17157, 25, 77, 4567, 441, 18423, 144, 77, 4567, 246, 5343, 25, 15894, 16539, 91, 13849, 3121, 25, 15894, 16539, 21388, 348, 16687, 13027, 25, 16687, 4577, 9885, 317, 81, 2339

In [31]:
texto_decodificado = tokenizador.decode(ids)
print(texto_decodificado)

A convivência com o ensino em uma escola pública, nos mostra \ xa0como a reforma do ensino médio é necessária, com o ensino precário, escolas quase desabando, falta de educadores, a formação de estudantes fica a \ xa0cada dia mais difícil., Atualmente o índice de jovens que procuram uma educação de qualidade é maior do que em todos os séculos, temos em vista que todos procuram a melhoria, tanto na estrutura da instituição como no ensino, muitos sonham em ser : médicos, professores, advogados, contadores, e assim sucessivamente., A solução seria a constatação de uma lei em prol dos estudantes, precisamos ser ouvidos, a voz dos jovens é a voz do futuro, queremos o ensino superior, queremos oportunidades em escolas públicas, escolas bem estruturada e profissionais competentes., Creio \ xa0que a reforma deveria ocorrer, pois o intuito é a melhoria ( Que tipo de melhoria?, \ xa0assim todos os jovens teriam a oportunidade de serem profissionais de sucesso. O incentivo ( aos jovens é o que to

In [32]:
input_codificado = tokenizador(texto, return_tensors='tf')
print(input_codificado)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


{'input_ids': <tf.Tensor: shape=(1, 271), dtype=int32, numpy=
array([[   11,    46,   270, 24522,  2475,   303,    91, 13849,   348,
          469,  5148,  5155,    25,  1125,  4201,    73,  5308,   759,
         5563,   231,    77,  8008,   246, 13849, 19640,   144,   539,
         2597,   206,  4354,    25,   303,    91, 13849,  1480,   415,
         2854,    25, 16687,  8774,  3151,  4948,   231,    25,  5988,
          203,  5725,  8294,   920,    25,    77, 11569,   203, 22157,
        13510,    77,    73,  5308,   759,  6073,   608,   578,  7859,
           27,    25, 12242,    91, 20373,   203, 18423,   220, 21237,
          244,   469, 17937,   203, 16944,   144,  2695,   246,   220,
          348,  1692,   463, 16426,    25,  1802,   383,   348,  3416,
          220,  1692, 21237,   244,    77,  5915,   359,    25,  1813,
          230,  9885,   240, 21600,   314,   286, 13849,    25,  6455,
          385,  1948,   348,   510,    39, 17306,    25, 17575,   206,
           25, 

# Tokenizando a base de dados

In [33]:
def funcao_tokenizadora(dados_texto):
    return tokenizador(dados_texto['essay'], truncation=True)

In [34]:
dataset_tokenizado = dados_redacoes.map(funcao_tokenizadora, batched=True, remove_columns=['essay'])

Map:   0%|          | 0/3656 [00:00<?, ? examples/s]

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

In [35]:
dataset_tokenizado

DatasetDict({
    treino: Dataset({
        features: ['score', 'input_ids', 'attention_mask'],
        num_rows: 3656
    })
    teste: Dataset({
        features: ['score', 'input_ids', 'attention_mask'],
        num_rows: 914
    })
})

In [36]:
dataset_tokenizado['treino'].to_pandas()

,score,input_ids,attention_mask
0,4,"[11, 46, 270, 24522, 2475, 303, 91, 13849, 348...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,5,"[11, 60, 1497, 6656, 5124, 9019, 2782, 22218, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,8,"[11, 60, 290, 8465, 144, 3240, 1668, 25, 243, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,8,"[11, 60, 19131, 203, 15, 23380, 19311, 246, 28...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,5,"[11, 2567, 441, 17363, 206, 220, 578, 2989, 78...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
3651,5,"[11, 46, 5148, 1802, 469, 17870, 7326, 230, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3652,6,"[11, 60, 7234, 1280, 3543, 1802, 91, 2885, 203...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3653,4,"[11, 7855, 325, 4776, 4323, 1559, 14086, 13232...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3654,6,"[11, 60, 23602, 235, 326, 23949, 206, 203, 179...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [37]:
dataset_tokenizado['teste'].to_pandas()

,score,input_ids,attention_mask
0,7,"[11, 60, 9133, 18342, 2294, 220, 13606, 1295, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,8,"[11, 599, 3088, 1306, 25, 22568, 2284, 2251, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,4,"[11, 46, 15182, 20233, 16866, 2969, 14150, 81,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,7,"[11, 60, 10910, 303, 469, 7592, 1828, 21261, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,7,"[11, 1876, 91, 3222, 240, 1774, 9151, 359, 25,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
909,6,"[11, 3444, 91, 5764, 240, 452, 3543, 25, 77, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
910,4,"[11, 60, 4880, 1235, 2011, 3575, 144, 91, 2370...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
911,5,"[11, 24848, 98, 8544, 243, 6634, 206, 220, 243...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
912,9,"[11, 4433, 91, 22999, 22429, 14533, 3507, 304,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


# Aula 03 - Ajustando dados do modelo

Ajustando a variável alvo

Para realizar o treinamento do modelo, a variável alvo precisa estar no formato específico ClassLabel. Vamos realizar esta transformação na base de dados.

In [38]:
dataset_tokenizado['treino'].features['score']

Value('int64')

In [39]:
from datasets import ClassLabel

In [40]:
dataset_tokenizado['treino'].unique('score')

[4, 5, 8, 6, 10, 7, 1, 9, 3, 0, 2]

In [41]:
scores = ClassLabel(names=[str(i) for i in range(11)])

In [42]:
scores

ClassLabel(names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

In [45]:
def mapear_labels(dados):
    dados['label'] = scores.str2int(str(dados['score']))
    return dados

In [46]:
dataset_tokenizado = dataset_tokenizado.map(mapear_labels, remove_columns=['score'])

Map:   0%|          | 0/3656 [00:00<?, ? examples/s]

Map:   0%|          | 0/914 [00:00<?, ? examples/s]

In [47]:
dataset_tokenizado = dataset_tokenizado.cast_column('label', scores)

Casting the dataset:   0%|          | 0/3656 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/914 [00:00<?, ? examples/s]

In [48]:
dataset_tokenizado

DatasetDict({
    treino: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 3656
    })
    teste: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 914
    })
})

In [49]:
dataset_tokenizado['treino'].features['label']

ClassLabel(names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

# Preparando dados de treino e teste

Agora que já realizamos todas as transformações nos dados, vamos iniciar o modelo pré-treinado e preparar os dados de treino e validação que serão utilizados no treinamento do novo modelo.

Modelo base para a transferência de aprendizado e tokenização
*   https://huggingface.co/Geotrend/distilbert-base-pt-cased



In [50]:
from transformers import TFAutoModelForSequenceClassification

In [51]:
checkpoint_modelo = "Geotrend/distilbert-base-pt-cased"

In [52]:
id2label = {0: '0', 1: '1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9',10:'10'}
label2id = {valor : chave for chave, valor in id2label.items()}

In [54]:
modelo = TFAutoModelForSequenceClassification.from_pretrained(
    checkpoint_modelo,
    num_labels=dataset_tokenizado['treino'].features['label'].num_classes,
    id2label=id2label,
    label2id=label2id,
    from_pt=True
)

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSeq

In [55]:
dados_treino = modelo.prepare_tf_dataset(
    dataset_tokenizado['treino'],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizador,
)

dados_validacao = modelo.prepare_tf_dataset(
    dataset_tokenizado['teste'],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizador,
)
